<a href="https://colab.research.google.com/github/yyuyulm/PoeTree/blob/main/PoeTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install drawing library
!pip install Qahirah

#update spacy and language model
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [ ]:
#check library versions and language model version

!pip show Qahirah
!pip show spacy
!pip show en-core-web-trf

In [ ]:
#download poem files

!npx degit yyuyulm/PoeTree/poems poems --force

In [ ]:
#load libraries and model

from IPython.display import display_png
from google.colab import files
import numpy as np
import math
import qahirah as qah
from qahirah import CAIRO, Colour, Rect, Vector
import spacy
from spacy.tokens import Doc, Span, Token
nlp = spacy.load("en_core_web_trf")

In [ ]:
#define text parsing fuctions

def read_text(filename):
  with open(filename, "r") as f:
    content = f.read()
    #print(content)
    return content

def read_text_lines(filename):
  with open(filename, "r") as f:
    lines = f.readlines()
    #print(lines)
    return lines

def parse_depth(root):
  parse_depth_helper(root,0)

def parse_depth_helper(token, depth):
  token._.depth = depth
  for child in token.children:
    parse_depth_helper(child, depth + 1)

In [ ]:
#define drawing functions

def display(img) :
    display_png(img.to_png_bytes(), raw = True)

#debug color
#for key in Colour.x11 :
#    print (key)

def tag2color(tag):
  color = {
    'ADJ': Colour.x11['yellow green'].replace_hsva(v = 0.9),
    'ADP': Colour.x11["sky blue"],
    'ADV': Colour.x11["medium turquoise"],
    'AUX': Colour.x11["indian red"],
    'CCONJ': Colour.x11["dark magenta"],
    'DET': Colour.x11["Medium Purple"],
    'INTJ': Colour.x11["orange red"],
    'NOUN': Colour.x11["forest green"],
    'NUM': Colour.x11["tomato"],
    'PART': Colour.x11["gold"],
    'PRON': Colour.x11["light salmon"],
    'PROPN': Colour.x11["medium orchid"],
    'PUNCT': Colour.x11["Dark Cyan"],
    'SCONJ': Colour.x11["midnight blue"],
    'SYM': Colour.x11["gray"],
    'VERB': Colour.x11["maroon"],
    'X': Colour.x11["black"],
    'SPACE' : Colour.from_hsva((0,0,0,0))
    }
  return color[tag]

def lerp(a,b,amount):
  return a*(1-amount) + b*amount

def map(in_min,in_max,out_min,out_max,input):
  alpha = (input - in_min)/(in_max-in_min)
  output = out_min + (out_max-out_min) * alpha
  return output

def draw_branch(pos0,pos1,col0,col1,ctx,line_weight):
  pos_mid = Vector(
    pos1.x,
    (pos0.y + pos1.y)/2
    )
  gradient = qah.Pattern.create_linear(
  p0 = pos0,
  p1 = pos_mid,
  colour_stops = [(0, col0),(1, col1)]
  )
  
  ctx.set_line_cap(CAIRO.LINE_CAP_BUTT)
  ctx.set_line_width(line_weight)
  ctx.set_source_colour(col1)
  ctx.move_to(pos_mid)
  ctx.line_to(pos1)
  ctx.stroke()

  ctx.set_source(gradient)
  ctx.move_to(pos0)
  ctx.line_to(pos_mid)
  ctx.stroke()

In [ ]:
#load poem

#try "some feel rain", "the end of landscape", "thank you", "rain", and more
poem_name = "some feel rain"

lines = read_text_lines(f'/content/poems/{poem_name}.txt')
poem = "".join(lines[3:])
poem = poem.replace("\n", " ")
poem = poem.replace("\t", " ")

for i in range(3):
  poem = poem.replace("  ", " ")

doc = nlp(poem)

sentences = list(doc.sents)
print("sentence count: " + str(len(sentences)))

for sentence in sentences:
  print(sentence.text)

In [ ]:
#drawing


#parameters
line_weight = 5
alpha = 0.65
line_height = 32 #lower this is text overflows
font_size = 20

#initialize context
pix = qah.ImageSurface.create \
  (
    dimensions = (1920,1080),
    format = CAIRO.FORMAT_ARGB32
  )
dim = pix.dimensions
ctx = qah.Context.create(pix)

#background
ctx.set_source_colour(Colour.x11["old lace"])
ctx.paint()

#display poem

ctx.select_font_face("serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
ctx.set_font_size(font_size)

line_offset = Vector(1120, 1020)
for i in range(len(lines)-1,-1,-1):
  line = lines[i]
  line = line.replace("\n", " ")
  line = line.replace("\t", "    ")
  line_doc = nlp(line)
  line_length = ctx.text_extents(line)[4]
  color_stops = []
  for j in range(len(line_doc)):
    span = line_doc[0:j]
    token = line_doc[j]
    length_pre = ctx.text_extents(span.text_with_ws)[4]
    length_word = ctx.text_extents(token.text)[4]
    word_center = length_pre + length_word/2
    color = tag2color(token.pos_)
    color_stops.append((word_center/line_length, color))

    #w_offset = Vector(length_pre,0) + line_offset
    #ctx.move_to(w_offset)
    #ctx.set_source_colour(color)
    #ctx.show_text(token.text)
  gradient = qah.Pattern.create_linear(
      p0 = line_offset,
      p1 = line_offset + Vector(line_length,0),
      colour_stops= color_stops
  )
  ctx.set_source(gradient)
  ctx.move_to(line_offset)
  ctx.show_text(line)
  line_offset += Vector(0,-line_height)
  

#text manipulation
Token.set_extension("sent_i", default=True, force=True)
Token.set_extension("depth", default=True, force=True)

ctx.select_font_face("serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
ctx.set_font_size(6)

#draw tree for each sentence
for i in range(len(sentences)):
  sent = sentences[i]
  textLoc = Vector(dim[0]/2,100)
  sent_extents = ctx.text_extents(sent.text)

  #ctx.set_source_colour(Colour.x11["old lace"])
  #ctx.paint()

  #offset = Vector(-sent_extents[4]/2, sent_extents[5]/2)
  #ctx.move_to(textLoc + offset)
  #ctx.set_source_colour(Colour.x11["deepskyblue4"])
  #ctx.show_text(sent.text)

  #get word coordinate
  sent_len = len(sent)
  w_x = np.empty(sent_len)
  w_y = np.empty(sent_len)

  parse_depth(sent.root)

  for j in range(sent_len):
    span = sent[0:j]
    token = sent[j]
    token._.sent_i = j
    length_pre = ctx.text_extents(span.text_with_ws)[4]
    length_post = ctx.text_extents(token.text)[4]
    w_x[j] = -sent_extents[4]/2 + length_pre + length_post/2
    w_y[j] = token._.depth
  
  #transform coordinate
  w_x *= 1
  w_y += 0.5
  w_y = w_y/(w_y.max()+2)
  w_y = lerp(w_y,np.sqrt(w_y),0.5)
  w_y = map(1,0,-80,580,w_y)
  tree_max_height = w_y.min()
  if tree_max_height < 20:
      w_y = map(w_y.min(),580,20,580,w_y)
  w_y = map(0,640,-320,320,w_y)
  tree_width = w_x.max() - w_x.min()
  if  tree_width > 620:
    w_x *= 620/tree_width
  w_y *= dim[1]/640
  w_x *= dim[1]/640

  w_x += dim[1]/2
  w_y += dim[1]/2
  
  root_coord = Vector(dim.y/2, 605/640*dim.y)
  root_color = Colour.x11["Sienna"]

  #draw branches to buffer
  sent_pix = qah.ImageSurface.create \
    (
    dimensions = dim,
    format = CAIRO.FORMAT_ARGB32
    )
  sent_ctx = qah.Context.create(sent_pix)

  for k in range(sent_len):
    token = sent[k]
    if token._.depth == 0:
      pos = Vector(w_x[token._.sent_i],w_y[token._.sent_i])
      col = tag2color(token.pos_)
      draw_branch(pos,root_coord,col,root_color,sent_ctx,line_weight)
    else:
      if(token.pos_ != "PUNCT" or k+1 < sent_len):
        par = token.head
        pos0 = Vector(w_x[token._.sent_i],w_y[token._.sent_i])
        pos1 = Vector(w_x[par._.sent_i],w_y[par._.sent_i])
        col0 = tag2color(token.pos_)
        col1 = tag2color(par.pos_)
        draw_branch(pos0,pos1,col0,col1,sent_ctx,line_weight)

  #display(sent_pix)

  #draw buffer to main
  pattern = qah.Pattern.create_for_surface(sent_pix)
  ctx.set_source(pattern)
  ctx.paint_with_alpha(alpha)

  #pix.write_to_png("/content/images/sentence "+str(i)+".png")
  #files.download("/content/images/sentence "+str(i)+".png")
  
display(pix)

In [ ]:
#save image
pix.write_to_png(f"{poem_name}.png")
files.download(f"{poem_name}.png")